In [ ]:
# 필수 라이브러리 임포트
from typing import Tuple
import matplotlib
import meep as mp
import nlopt
import numpy as np
import csv

# matplotlib의 백엔드를 "agg"로 설정하여 이미지를 파일로 저장 가능하게 합니다.
matplotlib.use("agg")
import matplotlib.pyplot as plt
 
# Lorentzian 함수 정의: Lorentzian 파라미터를 사용하여 복소 유전율 프로파일을 계산
def lorentzfunc(p: np.ndarray, x: np.ndarray) -> np.ndarray:
    N = len(p) // 3  # 파라미터가 3개씩 묶인 Lorentzian 항의 개수
    y = np.zeros(len(x))  # 결과값을 저장할 배열
    for n in range(N):
        A_n = p[3 * n + 0]  # 첫 번째 파라미터 (진폭)
        x_n = p[3 * n + 1]  # 두 번째 파라미터 (중심 주파수)
        g_n = p[3 * n + 2]  # 세 번째 파라미터 (감쇠율)
        y = y + A_n / (np.square(x_n) - np.square(x) - 1j * x * g_n)  # Lorentzian 함수 계산
    return y

# 잔차 함수 정의: 실제 값과 Lorentzian 모델 간의 차이를 계산하고 그라디언트도 반환
def lorentzerr(p: np.ndarray, x: np.ndarray, y: np.ndarray, grad: np.ndarray) -> float:
    N = len(p) // 3  # Lorentzian 항의 개수
    yp = lorentzfunc(p, x)  # 예측된 유전율 프로파일 계산
    val = np.sum(np.square(abs(y - yp)))  # 실제 값과 예측 값의 차이(L2 norm)

    # 그라디언트 계산
    for n in range(N):
        A_n = p[3 * n + 0]
        x_n = p[3 * n + 1]
        g_n = p[3 * n + 2]
        d = 1 / (np.square(x_n) - np.square(x) - 1j * x * g_n)  # Lorentzian 함수의 도함수
        if grad.size > 0:
            grad[3 * n + 0] = 2 * np.real(np.dot(np.conj(yp - y), d))  # 진폭에 대한 그라디언트
            grad[3 * n + 1] = -4 * x_n * A_n * np.real(np.dot(np.conj(yp - y), np.square(d)))  # 중심 주파수에 대한 그라디언트
            grad[3 * n + 2] = -2 * A_n * np.imag(np.dot(np.conj(yp - y), x * np.square(d)))  # 감쇠율에 대한 그라디언트
    return val  # 최종 오차 반환

# Lorentzian 파라미터 최적화 함수 정의
def lorentzfit(
    p0: np.ndarray,
    x: np.ndarray,
    y: np.ndarray,
    alg=nlopt.LD_LBFGS,
    tol: float = 1e-25,
    maxeval: float = 10000,
) -> Tuple[np.ndarray, float]:
    # NLopt 최적화 설정
    opt = nlopt.opt(alg, len(p0))  # 최적화 알고리즘 설정 (LD_LBFGS는 LBFGS 알고리즘)
    opt.set_ftol_rel(tol)  # 상대 오차 허용 범위 설정
    opt.set_maxeval(maxeval)  # 최대 반복 횟수 설정
    opt.set_lower_bounds(np.zeros(len(p0)))  # 하한값 설정
    opt.set_upper_bounds(float("inf") * np.ones(len(p0)))  # 상한값 설정
    opt.set_min_objective(lambda p, grad: lorentzerr(p, x, y, grad))  # 목표 함수 설정
    local_opt = nlopt.opt(nlopt.LD_LBFGS, len(p0))  # 로컬 최적화 설정
    local_opt.set_ftol_rel(1e-10)  # 로컬 최적화 상대 오차 설정
    local_opt.set_xtol_rel(1e-8)  # 로컬 최적화 X 변화 상대 오차 설정
    opt.set_local_optimizer(local_opt)  # 로컬 최적화 설정 추가
    popt = opt.optimize(p0)  # 최적화 실행
    minf = opt.last_optimum_value()  # 마지막 최적화 값 반환
    return popt, minf  # 최적화된 파라미터와 최소 오차 반환

In [54]:
# 메인 코드 실행 부분
# SiN.csv 파일에서 복소 유전율 데이터를 불러옴 (wavelength, real(n), imag(n) 순)
mydata = np.genfromtxt("Al_palik_data.csv", delimiter=",")
n = mydata[:, 1] + 1j * mydata[:, 2]  # 복소수 형태로 유전율 데이터 생성

In [55]:
# 유전율의 무한 주파수에서의 값 설정 (eps_inf)
eps_inf = 1
eps = np.square(n) - eps_inf  # 복소 유전율 프로파일 계산

In [56]:
# 주어진 파장 범위로 데이터를 필터링
wl = mydata[:, 0]*1e6
wl_min = 0.399  # 최소 파장 (단위: nm)
wl_max = 0.701  # 최대 파장 (단위: nm)

In [57]:
start_idx = np.where(wl > wl_min)  # 최소 파장 이상인 인덱스 찾기
start_idx
idx_start = start_idx[0][0]

In [58]:
end_idx = np.where(wl < wl_max)  # 최대 파장 이하인 인덱스 찾기
idx_end = end_idx[0][-1] + 1

In [69]:
# 주파수(f)로 변환 (주파수는 1/파장으로 계산)
freqs = 10 / wl  # 단위: 1/μm

In [70]:
freqs_reduced = freqs[idx_start:idx_end]  # 필터링된 주파수
wl_reduced = wl[idx_start:idx_end]  # 필터링된 파장
eps_reduced = eps[idx_start:idx_end]  # 필터링된 유전율

# 사용할 Lorentzian 항의 개수 (2개로 설정)
num_lorentzians = 2

# 랜덤 초기값을 여러 번 반복하여 최적화
num_repeat = 30
ps = np.zeros((num_repeat, 3 * num_lorentzians))
mins = np.zeros(num_repeat)
for m in range(num_repeat):
    p_rand = [10 ** (np.random.random()) for _ in range(3 * num_lorentzians)]  # 랜덤 초기값 생성
    ps[m, :], mins[m] = lorentzfit(p_rand, freqs_reduced, eps_reduced, nlopt.LD_MMA, 1e-25, 50000)
    print(f"iteration: {m:3d}, error: {mins[m]:.6f}")

# 최적화된 파라미터 출력
idx_opt = np.where(np.min(mins) == mins)[0][0]
print(f"optimal: {mins[idx_opt]:.6f}")

# 최적화된 파라미터로 'Medium' 객체 생성
E_susceptibilities = []
for n in range(num_lorentzians):
    mymaterial_freq = ps[idx_opt][3 * n + 1]
    mymaterial_gamma = ps[idx_opt][3 * n + 2]
    if mymaterial_freq == 0:
        mymaterial_sigma = ps[idx_opt][3 * n + 0]
        E_susceptibilities.append(mp.DrudeSusceptibility(frequency=1.0, gamma=mymaterial_gamma, sigma=mymaterial_sigma))
    else:
        mymaterial_sigma = ps[idx_opt][3 * n + 0] / mymaterial_freq**2
        E_susceptibilities.append(mp.LorentzianSusceptibility(frequency=mymaterial_freq, gamma=mymaterial_gamma, sigma=mymaterial_sigma))
mymaterial = mp.Medium(epsilon=eps_inf, E_susceptibilities=E_susceptibilities)

# 실제 데이터와 적합된 데이터 비교 그래프 출력
mymaterial_eps = [mymaterial.epsilon(f)[0][0] for f in freqs_reduced]
fig, ax = plt.subplots(ncols=2)
ax[0].plot(wl_reduced, np.real(eps_reduced) + eps_inf, "b-", label="actual")
ax[0].plot(wl_reduced, np.real(mymaterial_eps), "ro", markersize=3, label="fit")
ax[0].set_xlabel("wavelength (nm)")
ax[0].set_ylabel(r"real($\epsilon$)")
ax[0].legend()

ax[1].plot(wl_reduced, np.imag(eps_reduced), "b-", label="actual")
ax[1].plot(wl_reduced, np.imag(mymaterial_eps), "ro", markersize=3, label="fit")
ax[1].set_xlabel("wavelength (nm)")
ax[1].set_ylabel(r"imag($\epsilon$)")
ax[1].legend()

fig.suptitle(f"Comparison of Actual Material Data and Fit\nusing Drude-Lorentzian Susceptibility")
fig.subplots_adjust(wspace=0.3)
fig.savefig("eps_fit_sample.png", dpi=150, bbox_inches="tight")

iteration:   0, error: 763011.991701
iteration:   1, error: 32282.826806
iteration:   2, error: 32282.826806
iteration:   3, error: 32282.826806
iteration:   4, error: 32282.826806
iteration:   5, error: 825.814864
iteration:   6, error: 433292.925150
iteration:   7, error: 762995.652116
iteration:   8, error: 918660.905451
iteration:   9, error: 433362.485311
iteration:  10, error: 32282.826806
iteration:  11, error: 32155.927864
iteration:  12, error: 32209.108748
iteration:  13, error: 859034.900831
iteration:  14, error: 32282.826806
iteration:  15, error: 994008.652092
iteration:  16, error: 932496.927030
iteration:  17, error: 433857.285886
iteration:  18, error: 32282.826806
iteration:  19, error: 31988.812283
iteration:  20, error: 32282.826806
iteration:  21, error: 882717.626732
iteration:  22, error: 763010.919092
iteration:  23, error: 434633.434069
iteration:  24, error: 12148.353948
iteration:  25, error: 32282.826806
iteration:  26, error: 32209.108404
iteration:  27, er

In [63]:
# CSV 파일에는 각 파장에 대해 다음 5개의 열을 저장합니다:
# wavelength (nm), Re(eps_actual), Im(eps_actual), Re(eps_fit), Im(eps_fit)
csv_filename = "Al_palik_data_fit.csv"
with open(csv_filename, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    # 헤더 작성
    writer.writerow(["wavelength (nm)", "Re(eps_actual)", "Im(eps_actual)", "Re(eps_fit)", "Im(eps_fit)"])
    # 각 데이터 포인트에 대해 행(row) 작성
    for wl_val, eps_act, eps_fit in zip(wl_reduced, eps_reduced, mymaterial_eps):
        writer.writerow([
            wl_val,
            np.real(eps_act) + eps_inf,  # 실제 데이터에 eps_inf를 더해줍니다.
            np.imag(eps_act),
            np.real(eps_fit),
            np.imag(eps_fit)
        ])

print(f"CSV 파일 '{csv_filename}'로 저장되었습니다.")

CSV 파일 'Al_palik_data_fit.csv'로 저장되었습니다.
